# POKEBRO

**Autores:**

- Miguel Ángel Abalde Paz
- Oscar Fernández Argiz
- Borja Díaz Vázquez
- Javier Pérez Vázquez

# Introducción de la aplicación



> El siguiente manual de usuario replica el funcionamiento de nuestra aplicación, en este manual veremos tanto un sistema de recomendación de Pokemon como un sistema de valoración automática y valoración manual.

**Sistema de recomendación**

> El sistema de recomendación realizará varios procesos, entre ellos la tokenización de las descripiones de los Pokemon para su posterior creación del BagOfWords y de la matriz de distancias que serán usados para la recomendación de varios Pokemon a partir de una busqueda inicial.

**Sistema de valoración** 
> El sistema de valoración está compuesto por dos partes, la valoración manual que se basa en un formulario en el que el usuario puedes escribir sus valoraciones y la valoración automática en la que haremos uso de unos datos de entrenamiento para entrenar nuestro algoritmo de manera que podamos realizar un analisis de sentimientos de distintas opiniones sobre Pokemon. Para simular la existencia de multiples comentarios sobre Pokemon hemos generado un csv a partir de una busqueda en twitter.



# SISTEMA DE RECOMENDACIÓN



El primer paso para crear nuestro sistema de recomendación será la importación de distintas herramientas que nos permitan empezar a trabajar. 

Importamos **pandas** para trabajar con nuestros CSV a través de Dataframes, además de importar la herramienta **nltk** que nos ayudará en la tarea de tokenización de las descripciones.

In [48]:
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

El primer paso será el de cargar nuestro csv, en este caso **pokemon_info.csv**. Este csv contiene a todos los Pokemon existentes además de estadísticas sobre ellos y una descripción de la Pokedex como se puede observar en la tabla.

In [49]:
rawData = pd.read_csv('pokemon_info.csv')
rawData

,id,identifier,height,weight,base_experience,order,is_default,flavor_text
0,1,bulbasaur,7,69,64,1,1,A strange seed was\nplanted on its\nback at bi...
1,2,ivysaur,10,130,142,2,1,"When the bulb on\nits back grows\nlarge, it ap..."
2,3,venusaur,20,1000,236,3,1,The plant blooms\nwhen it is\nabsorbing solar ...
3,4,charmander,6,85,62,5,1,"Obviously prefers\nhot places. When\nit rains,..."
4,5,charmeleon,11,190,142,6,1,"When it swings\nits burning tail,\nit elevates..."
...,...,...,...,...,...,...,...,...
713,714,noibat,0,0,0,775,1,They live in pitch-black caves. Their enormous...
714,715,noivern,0,0,0,776,1,They fly around on moonless nights\nand attack...
715,716,xerneas,0,0,0,777,1,Legends say it can share eternal life.\nIt sle...
716,717,yveltal,0,0,0,778,1,When this legendary Pokémon’s wings and\ntail ...


El siguiente paso será la tokenización de las descripciones, para ello debemos recorrer nuestro dataframe. En cada iteración obtendremos la información de la columna *flavor_text* de nuestro dataframe y le aplicaremos la tokenización fila por fila, de manera que el resultado será una nueva columna en el dataframe *processed_text* que contendrá el texto procesado.

In [50]:
ps = PorterStemmer()
preText = []

i=0;
for row in rawData.itertuples():

    text = word_tokenize(rawData.iloc[i][7])
    stops = set(stopwords.words("english"))
    text = [ps.stem(w) for w in text if not w in stops and w.isalnum()]
    text = " ".join(text)
    
    preText.append(text)
    i=i+1
preData = rawData
preData['processed_text'] = preText

preData

,id,identifier,height,weight,base_experience,order,is_default,flavor_text,processed_text
0,1,bulbasaur,7,69,64,1,1,A strange seed was\nplanted on its\nback at bi...,A strang seed plant back birth the plant sprou...
1,2,ivysaur,10,130,142,2,1,"When the bulb on\nits back grows\nlarge, it ap...",when bulb back grow larg appear lose abil stan...
2,3,venusaur,20,1000,236,3,1,The plant blooms\nwhen it is\nabsorbing solar ...,the plant bloom absorb solar energi It stay mo...
3,4,charmander,6,85,62,5,1,"Obviously prefers\nhot places. When\nit rains,...",obvious prefer hot place when rain steam said ...
4,5,charmeleon,11,190,142,6,1,"When it swings\nits burning tail,\nit elevates...",when swing burn tail elev temperatur unbear hi...
...,...,...,...,...,...,...,...,...,...
713,714,noibat,0,0,0,775,1,They live in pitch-black caves. Their enormous...,they live cave their enorm ear emit ultrason w...
714,715,noivern,0,0,0,776,1,They fly around on moonless nights\nand attack...,they fli around moonless night attack careless...
715,716,xerneas,0,0,0,777,1,Legends say it can share eternal life.\nIt sle...,legend say share etern life It slept thousand ...
716,717,yveltal,0,0,0,778,1,When this legendary Pokémon’s wings and\ntail ...,when legendari pokémon wing tail feather sprea...


Ejemplo de descripción sin tokenizar.

In [51]:
preData.iloc[0][7]


'A strange seed was\nplanted on its\nback at birth.\x0cThe plant sprouts\nand grows with\nthis POKéMON.'

Ejemplo de descripción tokenizada.

In [52]:
preData.iloc[0][8]

'A strang seed plant back birth the plant sprout grow pokémon'

Una vez tokenizadas las descripciones creamos el *BagOfWords*. Para la creación del BagOfWords importamos la libreria **TfidfVectorizer** que contendrá las herramientas necesarias para su creación.

El BagOfWords contendrá las palabras contenidas en las descripciones tokenizadas y su frecuencia de aparición en todas ellas. Para ello creamos el BagOfWords llamando a TfidfVectorizer() y posteriormente hacemos el fit sobre la columna de texto procesado y realizamos el transform sobre la misma columna lo que nos resulta en nuestra variable textsBoW que contiene lo descrito anteriormente.

In [53]:
from sklearn.feature_extraction.text import TfidfVectorizer

bagOfWordsModel = TfidfVectorizer()
bagOfWordsModel.fit(preData['processed_text'])
textsBoW= bagOfWordsModel.transform(preData['processed_text'])
print("Finished")

Finished


In [54]:
textsBoW.shape

(718, 3017)

Mostramos el contenido de nuestra variable **textsBoW**

In [55]:
print(textsBoW)

  (0, 2630)	0.1813053884156978
  (0, 2499)	0.3573407812365763
  (0, 2457)	0.3573407812365763
  (0, 2236)	0.3573407812365763
  (0, 1910)	0.12021381063868418
  (0, 1883)	0.5713665604880965
  (0, 1165)	0.25554053485302297
  (0, 266)	0.34147466227401224
  (0, 199)	0.24745512257122929
  (1, 2921)	0.21617797692488688
  (1, 2463)	0.3095255010078092
  (1, 1483)	0.30289507729672716
  (1, 1440)	0.3170424554392978
  (1, 1415)	0.30289507729672716
  (1, 1230)	0.38756491433490514
  (1, 1165)	0.26083173549586963
  (1, 348)	0.38756491433490514
  (1, 199)	0.2525789073530748
  (1, 124)	0.23702762759148438
  (1, 19)	0.2915986140166228
  (2, 2630)	0.18617906553178423
  (2, 2542)	0.3506538557982136
  (2, 2471)	0.3113980855493526
  (2, 2387)	0.38990962604707463
  (2, 2238)	0.36694647251456575
  :	:
  (716, 2941)	0.21033961302520698
  (716, 2932)	0.2825961560092591
  (716, 2921)	0.18182748490483877
  (716, 2583)	0.18782899352928542
  (716, 2452)	0.3259811874146669
  (716, 2073)	0.3259811874146669
  (716, 191

Mostramos las palabras contenidas en nuestro BagOfWords.

In [56]:
bagOfWordsModel.get_feature_names()

['000',
 '10',
 '100',
 '1000',
 '10000',
 '108',
 '160',
 '1600',
 '24',
 '300',
 '328',
 '33',
 '390',
 '400',
 '48',
 '50',
 '60',
 'abandon',
 'abdomen',
 'abil',
 'abl',
 'abomin',
 'absol',
 'absorb',
 'acceler',
 'accept',
 'accid',
 'accul',
 'accumul',
 'achiev',
 'achèv',
 'acid',
 'acier',
 'across',
 'act',
 'action',
 'activ',
 'actual',
 'acéré',
 'ad',
 'adapt',
 'addit',
 'adept',
 'adhésif',
 'adjust',
 'admir',
 'ador',
 'adroitli',
 'adversair',
 'affaibl',
 'affect',
 'affection',
 'afford',
 'aflamanoir',
 'afraid',
 'after',
 'age',
 'aggress',
 'aggron',
 'agil',
 'agilité',
 'agir',
 'agit',
 'ago',
 'aigu',
 'aiguis',
 'aiguisé',
 'ail',
 'aim',
 'ainsi',
 'air',
 'airborn',
 'alarm',
 'alentour',
 'alert',
 'aliment',
 'all',
 'alli',
 'allow',
 'allègrement',
 'almost',
 'aloft',
 'alon',
 'along',
 'alpha',
 'also',
 'altaria',
 'although',
 'alway',
 'ami',
 'among',
 'amortiss',
 'amount',
 'amperag',
 'amplifi',
 'amus',
 'amèn',
 'amèr',
 'an',
 'anchwat

Una vez creado el BagOfWords el siguiente paso será crear la matriz de distancias. Para su creación importaremos **pairwise_distances**.

La matriz de distancia usará la métrica *manhattan* para determinar la distancia entre los elementos. Al realizar el calculo de distancias entre los elementos lo que hace esta librería con la metrica utilizada es crear una matriz que contendrá todos los Pokemon de nuestro csv y además estos estarán colocados entre ellos según su parecido. 



In [57]:
from sklearn.metrics import pairwise_distances

distance_matrix= pairwise_distances(textsBoW,textsBoW ,metric='manhattan')

In [59]:
print(distance_matrix.shape)

(718, 718)


El siguiente paso será indicarle un valor de busqueda inicial en este caso Raikou. Además mostramos su posición en el csv, en nuestro caso **242**

In [60]:
searchTitle = "raikou"
indexOfTitle = preData[preData['identifier']==searchTitle].index.values[0]
indexOfTitle

242

Una vez hemos introducido el elemento de busqueda inicial crearemos la variable *distance_scores* que contendrá la puntuación de cada elemento con respecto a nuestro valor de busqueda inicial *list(enumerate(distance_matrix[indexOfTitle]))*. Cada uno de los elementos puede tener una puntuación del 0 al 10 debido a la métrica Manhattan, siendo 0 lo mas parecido y 10 lo mas alejado. Como podemos observar al mostrar *distance_scores* el elemento 242 tiene una puntuación de 0 ya que es el propio elemento que se ha usado como busqueda inicial.

In [61]:
distance_scores = list(enumerate(distance_matrix[indexOfTitle]))
distance_scores

[(0, 5.6960928935490855),
 (1, 6.53266179389573),
 (2, 6.107544125453981),
 (3, 5.846007392984899),
 (4, 6.216353106880289),
 (5, 5.8487948224955515),
 (6, 6.374623806044722),
 (7, 6.825656272843689),
 (8, 5.999204376169063),
 (9, 6.498006147481873),
 (10, 6.272822863510913),
 (11, 6.564596437105244),
 (12, 6.347554363627411),
 (13, 6.000754046933665),
 (14, 6.371031534827554),
 (15, 6.6457382704308445),
 (16, 6.181609110499572),
 (17, 6.518950473934029),
 (18, 6.240333001991423),
 (19, 6.037897001856137),
 (20, 6.641336037145418),
 (21, 6.399609796413255),
 (22, 6.232505409030078),
 (23, 5.923046065865427),
 (24, 5.706486082532058),
 (25, 6.1557455665046525),
 (26, 6.498535100674753),
 (27, 5.83895097392474),
 (28, 6.107023618757751),
 (29, 6.0027172686143455),
 (30, 6.644250618512836),
 (31, 6.004904464717774),
 (32, 6.10518917742985),
 (33, 6.121570855153121),
 (34, 6.387234810706017),
 (35, 6.434077785168498),
 (36, 5.976814546091147),
 (37, 6.3778971685494925),
 (38, 6.55068976959

Una vez obtenidas las puntuaciones debemos ordenarlas para mostrar los 10 resultados mas parecidos. Obviamente hemos de omitir el primer elemento ya que es el propio elemento de la busqueda.

In [62]:
ordered_scores = sorted(distance_scores, key=lambda x: x[1])
ordered_scores

[(242, 0.0),
 (175, 4.955756377066811),
 (667, 4.980078553055622),
 (144, 5.094129296090852),
 (177, 5.154889759281152),
 (440, 5.216210756155929),
 (106, 5.268159255814101),
 (181, 5.314651096121145),
 (658, 5.444202702390927),
 (688, 5.541540227676474),
 (134, 5.551591859444804),
 (395, 5.584411964033811),
 (182, 5.590343025327073),
 (155, 5.595691505834516),
 (439, 5.616518749650636),
 (224, 5.662790708689686),
 (114, 5.6684626230841975),
 (212, 5.671105593342205),
 (187, 5.67591996949992),
 (390, 5.680341880244215),
 (122, 5.684239378609361),
 (413, 5.685487290614019),
 (0, 5.6960928935490855),
 (691, 5.6972153587756065),
 (24, 5.706486082532058),
 (465, 5.716286588492614),
 (702, 5.730545112124161),
 (703, 5.731081975614449),
 (492, 5.74443180837913),
 (706, 5.74735240251239),
 (479, 5.78091352488352),
 (128, 5.794166987680574),
 (223, 5.798312746123348),
 (394, 5.808193142860687),
 (446, 5.815411472927465),
 (414, 5.8200167504458316),
 (27, 5.83895097392474),
 (3, 5.8460073929848

Cogemos los 11 primeros elementos mas parecidos a Raikou y omitimos el primero como dijimos anteriormente.

In [63]:
top_scores = ordered_scores[1:11]
top_scores

[(175, 4.955756377066811),
 (667, 4.980078553055622),
 (144, 5.094129296090852),
 (177, 5.154889759281152),
 (440, 5.216210756155929),
 (106, 5.268159255814101),
 (181, 5.314651096121145),
 (658, 5.444202702390927),
 (688, 5.541540227676474),
 (134, 5.551591859444804)]

In [64]:
top_indexes = [i[0] for i in top_scores]
top_indexes

[175, 667, 144, 177, 440, 106, 181, 658, 688, 134]

Por ultimo accedemos a predata con los identificadores obtenidos en los top_indexes y mostramos el nombre correspondiente al elemento.

En este caso el resultado de recomendación con nuestro parametro de busqueda *Raikou* es el siguiente:

In [65]:
preData['identifier'].iloc[top_indexes]

175       togetic
667        pyroar
144        zapdos
177          xatu
440        chatot
106    hitmonchan
181     bellossom
658      bunnelby
688    barbaracle
134       jolteon
Name: identifier, dtype: object

# SISTEMA DE VALORACIÓN

**Valoración Automática**

Una vez hemos descrito nuestro sistema de recomendación daremos paso a describir nuestro sistema de valoración. 

En este sistema se importarán varias utilidades como en el sistema anterior.

Importamos pandas para trabajar con nuestros dataframes además de importar nuevamente la libreria nltk para la tokenización de las palabras.

Ademas cabe destacar el import de **train_test_split** que nos permitirá dividir un dataset en bloques para posteriormente usarlo como entrenamiento de nuestro algoritmo de valoración.

In [66]:
import numpy as np
import pandas as pd 

from subprocess import check_output

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

import nltk
from nltk.corpus import stopwords
from nltk.classify import SklearnClassifier

from subprocess import check_output

El siguiente paso como en el sistema de recomendación será la carga de nuestro CSV. En este caso *Sentiment.csv* es un csv que contiene distintas opiniones sacadas de Twitter con una columna de intención "Neutral", "Positive" o "Negative" que indica la intención del comentario. Este csv será utilizado para el entrenamiento de nuestro algoritmo de valoración automática.

In [67]:
data = pd.read_csv('Sentiment.csv')
data = data.head(13871)
data

,id,candidate,candidate_confidence,relevant_yn,relevant_yn_confidence,sentiment,sentiment_confidence,subject_matter,subject_matter_confidence,candidate_gold,name,relevant_yn_gold,retweet_count,sentiment_gold,subject_matter_gold,text,tweet_coord,tweet_created,tweet_id,tweet_location,user_timezone
0,1,No candidate mentioned,1.0000,yes,1.0000,Neutral,0.6578,None of the above,1.0000,NaN,I_Am_Kenzi,NaN,5,NaN,NaN,RT @NancyLeeGrahn: How did everyone feel about...,NaN,2015-08-07 09:54:46 -0700,629697200650592256,NaN,Quito
1,2,Scott Walker,1.0000,yes,1.0000,Positive,0.6333,None of the above,1.0000,NaN,PeacefulQuest,NaN,26,NaN,NaN,RT @ScottWalker: Didn't catch the full #GOPdeb...,NaN,2015-08-07 09:54:46 -0700,629697199560069120,NaN,NaN
2,3,No candidate mentioned,1.0000,yes,1.0000,Neutral,0.6629,None of the above,0.6629,NaN,PussssyCroook,NaN,27,NaN,NaN,RT @TJMShow: No mention of Tamir Rice and the ...,NaN,2015-08-07 09:54:46 -0700,629697199312482304,NaN,NaN
3,4,No candidate mentioned,1.0000,yes,1.0000,Positive,1.0000,None of the above,0.7039,NaN,MattFromTexas31,NaN,138,NaN,NaN,RT @RobGeorge: That Carly Fiorina is trending ...,NaN,2015-08-07 09:54:45 -0700,629697197118861312,Texas,Central Time (US & Canada)
4,5,Donald Trump,1.0000,yes,1.0000,Positive,0.7045,None of the above,1.0000,NaN,sharonDay5,NaN,156,NaN,NaN,RT @DanScavino: #GOPDebate w/ @realDonaldTrump...,NaN,2015-08-07 09:54:45 -0700,629697196967903232,NaN,Arizona
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13866,13867,No candidate mentioned,1.0000,yes,1.0000,Negative,0.7991,Abortion,0.6014,No candidate mentioned,SantoliDonato,yes,7,Negative,Abortion\nWomen's Issues (not abortion though),RT @cappy_yarbrough: Love to see men who will ...,NaN,2015-08-07 09:29:43 -0700,629690895479250944,Como,NaN
13867,13868,Mike Huckabee,0.9611,yes,1.0000,Positive,0.7302,None of the above,0.9229,Mike Huckabee,mhfa16hq,yes,1,NaN,NaN,RT @georgehenryw: Who thought Huckabee exceede...,NaN,2015-08-07 09:25:02 -0700,629689719056568320,USA,NaN
13868,13869,Ted Cruz,1.0000,yes,1.0000,Positive,0.8051,None of the above,0.9647,Ted Cruz,DrottM,yes,67,Positive\nNeutral,NaN,"RT @Lrihendry: #TedCruz As President, I will a...",NaN,2015-08-07 07:19:18 -0700,629658075784282112,NaN,NaN
13869,13870,Donald Trump,1.0000,yes,1.0000,Negative,1.0000,Women's Issues (not abortion though),0.9202,Donald Trump,danijeantheq,yes,149,NaN,Women's Issues (not abortion though),RT @JRehling: #GOPDebate Donald Trump says tha...,NaN,2015-08-07 09:54:04 -0700,629697023663546368,NaN,NaN


Antes de empezar a trabajar con el csv debemos darle el formato que queremos para adecuarlo a nuestras necesidades. En este caso cambiaremos el valor de la columna *Sentiment* sustituyendo los valores de texto por valores numéricos. 

La transformación será la siguiente:

- Positive = 1

- Neutral = 0

- Negative = -1

In [68]:
data = data[['text','sentiment']]
data.loc[data.sentiment == 'Neutral', 'sentiment'] = 0
data.loc[data.sentiment == 'Positive', 'sentiment'] = 1
data.loc[data.sentiment == 'Negative', 'sentiment'] = -1
data

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid d

,text,sentiment
0,RT @NancyLeeGrahn: How did everyone feel about...,0
1,RT @ScottWalker: Didn't catch the full #GOPdeb...,1
2,RT @TJMShow: No mention of Tamir Rice and the ...,0
3,RT @RobGeorge: That Carly Fiorina is trending ...,1
4,RT @DanScavino: #GOPDebate w/ @realDonaldTrump...,1
...,...,...
13866,RT @cappy_yarbrough: Love to see men who will ...,-1
13867,RT @georgehenryw: Who thought Huckabee exceede...,1
13868,"RT @Lrihendry: #TedCruz As President, I will a...",1
13869,RT @JRehling: #GOPDebate Donald Trump says tha...,-1


Mostramos la frecuencia de aparición de cada valor numérico.

In [69]:
data['sentiment'].value_counts()

-1    8493
 0    3142
 1    2236
Name: sentiment, dtype: int64

El siguiente paso será el de tokenizar los comentarios del csv para posteriormente poder entrenar nuestro algoritmo. El proceso de tokenización es muy parecido al explicado anteriormente, recorreremos nuestro dataframe y realizaremos la tokenización de la columna con indice 1 que corresponde a los comentarios. Su tokenización usará como idioma de referencia el inglés y añadirá una nueva columna a nuestro dataframe que será *processed_text*

In [70]:
ps = PorterStemmer()

preprocessedText = []

for row in data.itertuples():
    
    
    text = word_tokenize(row[1])
    ## Remove stop words
    stops = set(stopwords.words("english"))
    text = [ps.stem(w) for w in text if not w in stops and w.isalnum()]
    text = " ".join(text)
    
    preprocessedText.append(text)

preprocessedData = data
preprocessedData['processed_text'] = preprocessedText

preprocessedData

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,text,sentiment,processed_text
0,RT @NancyLeeGrahn: How did everyone feel about...,0,RT nancyleegrahn how everyon feel climat chang...
1,RT @ScottWalker: Didn't catch the full #GOPdeb...,1,RT scottwalk did catch full gopdeb last night ...
2,RT @TJMShow: No mention of Tamir Rice and the ...,0,RT tjmshow No mention tamir rice gopdeb held c...
3,RT @RobGeorge: That Carly Fiorina is trending ...,1,RT robgeorg that carli fiorina trend hour her ...
4,RT @DanScavino: #GOPDebate w/ @realDonaldTrump...,1,RT danscavino gopdeb realdonaldtrump deliv hig...
...,...,...,...
13866,RT @cappy_yarbrough: Love to see men who will ...,-1,RT love see men never face pregnanc talk I bod...
13867,RT @georgehenryw: Who thought Huckabee exceede...,1,RT georgehenryw who thought huckabe exceed exp...
13868,"RT @Lrihendry: #TedCruz As President, I will a...",1,RT lrihendri tedcruz As presid I alway tell tr...
13869,RT @JRehling: #GOPDebate Donald Trump says tha...,-1,RT jrehl gopdeb donald trump say time polit co...


El siguiente paso será la creación del BagOfWords de la misma manera que lo hemos creado en el sistema de recomendación.

In [71]:
from sklearn.feature_extraction.text import TfidfVectorizer

bagOfWordsModel = TfidfVectorizer()
bagOfWordsModel.fit(preprocessedData['processed_text'])
textsBoW= bagOfWordsModel.transform(preprocessedData['processed_text'])
print("Finished")

Finished


In [72]:
textsBoW.shape

(13871, 11723)

Una vez creado el BagOfWords daremos paso al entrenamiento de nuestro algoritmo. Para ello definimos nuestro algoritmo clasificador SVM con el parametro **kernel=linear** que supone más flexibilidad en la asignación de penalizaciones y funciones de coste.

Para el entrenamiento de nuestro algoritmo debemos definir 2 variables que son:

- X_Train: Contiene una lista de valoraciones.

- Y_Train: Contiene el sentimiento de cada una de las valoraciones, es decir positivo, neutro o negativo.

Estas dos variables se le pasarán al algoritmo con fit y este asociará ciertos patrones para el analisis de sentimientos de una frase o comentario.

In [73]:
from sklearn import svm
svc = svm.SVC(kernel='linear')

X_train = textsBoW #Documentos
Y_train = data['sentiment']
Y_train=Y_train.astype('int')
svc.fit(X_train, Y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

Cargamos el csv que usaremos para testear nuestro algoritmo de valoración automatica. Este csv contendrá una serie de comentarios de los cual nuestro algoritmo deberá predecir su sentimiento.

In [74]:
testData = pd.read_csv('semeval-2017-test.csv', delimiter='	')
testData = testData.head(1000)
testData

,label,text
0,0,Trump is building a wall on the Mexican border...
1,-1,@lasinferencias & the WALL Trump wants to buil...
2,-1,President Elect? More like President Erect! A ...
3,0,"Ok, I know a lot of you think a wall on the Me..."
4,0,The Great Mexican Wall Deception: Trump's Amer...
...,...,...
995,-1,Listening to Melania practice the speech Miche...
996,-1,is there anyone worse than Grayson Allen
997,1,Grayson Allen is good. I think he can pay next...
998,1,We need to start talking more about how Grayso...


En este caso tambien debemos realizar la tokenización del texto para posteriormente pasarselas a nuestro algoritmo. El proceso es el mismo que para los casos anteriores.

In [75]:
ps = PorterStemmer()

preprocessedText = []

for row in testData.itertuples():
    
    
    text = word_tokenize(row[2])
    ## Remove stop words
    stops = set(stopwords.words("english"))
    text = [ps.stem(w) for w in text if not w in stops and w.isalnum()]
    text = " ".join(text)
    
    preprocessedText.append(text)

preprocessedDataTest = testData
preprocessedDataTest['processed_text'] = preprocessedText

preprocessedDataTest

,label,text,processed_text
0,0,Trump is building a wall on the Mexican border...,trump build wall mexican border stop herrion c...
1,-1,@lasinferencias & the WALL Trump wants to buil...,lasinferencia wall trump want build I research...
2,-1,President Elect? More like President Erect! A ...,presid elect more like presid erect A wall On ...
3,0,"Ok, I know a lot of you think a wall on the Me...",Ok I know lot think wall mexican border insan ...
4,0,The Great Mexican Wall Deception: Trump's Amer...,the great mexican wall decept trump america al...
...,...,...,...
995,-1,Listening to Melania practice the speech Miche...,listen melania practic speech michel obama wro...
996,-1,is there anyone worse than Grayson Allen,anyon wors grayson allen
997,1,Grayson Allen is good. I think he can pay next...,grayson allen good I think pay next level 2nd ...
998,1,We need to start talking more about how Grayso...,We need start talk grayson allen play point gu...


Mostramos el numero de valores negativos, positivos y neutros

In [76]:
testData['label'].value_counts()

 0    474
-1    341
 1    185
Name: label, dtype: int64

Creamos la variable textsBoWTest con el transform del BagOfWords sobre la columna del texto procesado.

In [77]:
textsBoWTest= bagOfWordsModel.transform(preprocessedDataTest['processed_text'])
print("Finished")

Finished


In [78]:
textsBoWTest.shape

(1000, 11723)

Una vez obtenemos el textsBoWTest lo igualamos a la variable X_test que le pasaremos al algoritmo para que realice la predicción sobre las descripciones.

In [79]:
X_test = textsBoWTest


predictions = svc.predict(X_test)
print("Finished")

Finished


Mostramos las predicciones obtenidas:

In [80]:
print(predictions)

[-1 -1 -1 -1 -1 -1 -1 -1 -1 -1  0 -1  0 -1 -1 -1 -1 -1 -1  0  1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1  0 -1  0 -1  0  1  0 -1  0 -1 -1 -1  1 -1 -1
 -1 -1 -1  0 -1 -1 -1 -1  0 -1 -1 -1  0 -1 -1 -1 -1 -1  0  0  0  0  0 -1
 -1  0 -1  0  0  0  0 -1 -1  0 -1  0  0 -1 -1 -1  0 -1  0 -1  0  0 -1 -1
 -1 -1 -1 -1 -1 -1  0  0 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1  0  0  0
 -1 -1 -1 -1 -1 -1  0  0  0 -1  0 -1 -1 -1 -1 -1 -1 -1 -1 -1  1 -1 -1 -1
  0 -1 -1 -1  0 -1 -1 -1 -1  0  0  1 -1  0 -1 -1 -1 -1  1 -1  0 -1 -1 -1
 -1  1 -1 -1  0 -1 -1  0  0  0 -1 -1  0  0 -1 -1 -1 -1 -1  1 -1  0  1 -1
 -1 -1 -1 -1  0  0 -1 -1 -1  0 -1 -1 -1 -1  0  0 -1 -1 -1 -1  0 -1 -1 -1
 -1 -1 -1 -1 -1  0 -1 -1  0  0 -1 -1  0  1  0  0  0 -1 -1 -1  0 -1  0  0
  0 -1 -1 -1 -1 -1 -1 -1 -1 -1  0 -1 -1 -1  0 -1 -1 -1 -1 -1 -1  0 -1 -1
 -1 -1 -1  0  0  0 -1 -1 -1  0 -1 -1  0 -1 -1 -1 -1 -1  1  0 -1  0 -1 -1
 -1 -1  0 -1 -1 -1 -1 -1 -1 -1  0 -1 -1  0 -1 -1 -1 -1  0  0 -1 -1 -1 -1
 -1  0  0  1 -1 -1 -1  0 -1 -1 -1 -1 -1 -1  0 -1 -1

El siguiente paso será el de valorar la predicción que ha realizado nuestro algoritmo para ello usaremos la herramienta **classification_report**, esta herramienta nos permite obtener unas estadisticas acerca de la predicción realizada. Para ello debemos llamar a classification_report con los argumentos Y_test, que corresponde a la verdadera intención de las descripciones analizadas, y predictions que contendrá las predicciones realizadas.

Como podemos observar el ratio de acierto es bastante bajo, esto se debe a que nuestros datos de entrenamiento eran bastante escasos. Si aumentamos el numero de datos de entrenamiento tambien subirá nuestro indice de aciertos.

In [81]:
from sklearn.metrics import classification_report

Y_test = testData['label']

print (classification_report(Y_test, predictions))

              precision    recall  f1-score   support

          -1       0.41      0.88      0.56       341
           0       0.57      0.29      0.39       474
           1       0.52      0.08      0.13       185

    accuracy                           0.45      1000
   macro avg       0.50      0.42      0.36      1000
weighted avg       0.51      0.45      0.40      1000



Por último y como pequeña simulación de nuestro sistema en concreto realizaremos otra predicción pero esta vez sobre un csv de opiniones de Pokemon obtenido de mensajes de Twitter. El procedimiento a seguir para su predicción es el mismo que el seguido por el csv anterior por lo que no entraremos en detalles.

Carga del csv.

In [82]:
pokemonDataComments = pd.read_csv('PokeInfoDef.csv')
pokemonDataComments["text"]= pokemonDataComments["text"].map(str)
pokemonDataComments

,tweetDate,twitterId,handle,text,profileUrl,name,tweetLink,timestamp,query
0,Sat Jan 23 16:39:38 +0000 2021,934948599028502528,PeaceOfYoshi,"Meganium best... cause, well ive explained it ...",https://twitter.com/PeaceOfYoshi,The True Yoshi-Chara,https://twitter.com/PeaceOfYoshi/status/135301...,2021-01-26T18:54:56.280Z,https://twitter.com/search?q=lang%3Aen%20pokem...
1,Fri Jan 22 00:01:19 +0000 2021,43285703,docvalentine,@nosplendorr hmmm based on his pokemon opinion...,https://twitter.com/docvalentine,Doc Valentine,https://twitter.com/docvalentine/status/135240...,2021-01-26T18:54:56.280Z,https://twitter.com/search?q=lang%3Aen%20pokem...
2,Sat Jan 23 16:56:15 +0000 2021,714849904888299521,StealthHawkDude,@PeaceOfYoshi i see we differ in a lot of poke...,https://twitter.com/StealthHawkDude,StealthHawk,https://twitter.com/StealthHawkDude/status/135...,2021-01-26T18:54:56.280Z,https://twitter.com/search?q=lang%3Aen%20pokem...
3,Sun Jan 24 08:35:16 +0000 2021,193929402,KaggyFilms,Thats gonna be a yikes from me Cotton.,https://twitter.com/KaggyFilms,Alejandro Saab 🏀,https://twitter.com/KaggyFilms/status/13532600...,2021-01-26T18:54:56.280Z,https://twitter.com/search?q=lang%3Aen%20pokem...
4,Sat Jan 23 07:24:07 +0000 2021,929101255284285440,ItsAbouTimeJoey,Posted a Pokémon tier list. \n\nGonna lose som...,https://twitter.com/ItsAbouTimeJoey,Joey ⏱,https://twitter.com/ItsAbouTimeJoey/status/135...,2021-01-26T18:54:56.280Z,https://twitter.com/search?q=lang%3Aen%20pokem...
...,...,...,...,...,...,...,...,...,...
576,Wed Oct 16 14:25:32 +0000 2019,719461958550822912,zaneflynt,imagine having real Pokémon opinions.. couldn'...,https://twitter.com/zaneflynt,hawke the nibblenib,https://twitter.com/zaneflynt/status/118447546...,2021-01-26T18:54:56.282Z,https://twitter.com/search?q=lang%3Aen%20pokem...
577,Fri Jul 14 16:46:29 +0000 2017,2267742470,Poijz,I liked a @YouTube video https://t.co/T31TIjz1...,https://twitter.com/Poijz,Dylan ポイ❄️,https://twitter.com/Poijz/status/8859033354753...,2021-01-26T18:54:56.282Z,https://twitter.com/search?q=lang%3Aen%20pokem...
578,Wed Oct 16 14:25:32 +0000 2019,719461958550822912,zaneflynt,imagine having real Pokémon opinions.. couldn'...,https://twitter.com/zaneflynt,hawke the nibblenib,https://twitter.com/zaneflynt/status/118447546...,2021-01-26T18:54:56.282Z,https://twitter.com/search?q=lang%3Aen%20pokem...
579,Sun Nov 03 02:45:47 +0000 2019,15121903,cheesegratersan,"me when it comes to ""Pokemon Opinions"": im a r...",https://twitter.com/cheesegratersan,hhhh,https://twitter.com/cheesegratersan/status/119...,2021-01-26T18:54:56.282Z,https://twitter.com/search?q=lang%3Aen%20pokem...


Tokenización de los comentarios.

In [83]:
psOpinions = PorterStemmer()

preprocessedPokeOpinions = []

for row in pokemonDataComments.itertuples():
    textOpinions = word_tokenize(row[4])
    stopsOpinions = set(stopwords.words("english"))
    textOpinions = [psOpinions.stem(w) for w in textOpinions if not w in stopsOpinions and w.isalnum()]
    textOpinions = " ".join(textOpinions)
    
    preprocessedPokeOpinions.append(textOpinions)

preprocessedDataOpinions = pokemonDataComments
preprocessedDataOpinions['processed_text'] = preprocessedPokeOpinions

preprocessedDataOpinions

,tweetDate,twitterId,handle,text,profileUrl,name,tweetLink,timestamp,query,processed_text
0,Sat Jan 23 16:39:38 +0000 2021,934948599028502528,PeaceOfYoshi,"Meganium best... cause, well ive explained it ...",https://twitter.com/PeaceOfYoshi,The True Yoshi-Chara,https://twitter.com/PeaceOfYoshi/status/135301...,2021-01-26T18:54:56.280Z,https://twitter.com/search?q=lang%3Aen%20pokem...,meganium best caus well ive explain
1,Fri Jan 22 00:01:19 +0000 2021,43285703,docvalentine,@nosplendorr hmmm based on his pokemon opinion...,https://twitter.com/docvalentine,Doc Valentine,https://twitter.com/docvalentine/status/135240...,2021-01-26T18:54:56.280Z,https://twitter.com/search?q=lang%3Aen%20pokem...,nosplendorr hmmm base pokemon opinion say trus...
2,Sat Jan 23 16:56:15 +0000 2021,714849904888299521,StealthHawkDude,@PeaceOfYoshi i see we differ in a lot of poke...,https://twitter.com/StealthHawkDude,StealthHawk,https://twitter.com/StealthHawkDude/status/135...,2021-01-26T18:54:56.280Z,https://twitter.com/search?q=lang%3Aen%20pokem...,peaceofyoshi see differ lot pokemon opinion
3,Sun Jan 24 08:35:16 +0000 2021,193929402,KaggyFilms,Thats gonna be a yikes from me Cotton.,https://twitter.com/KaggyFilms,Alejandro Saab 🏀,https://twitter.com/KaggyFilms/status/13532600...,2021-01-26T18:54:56.280Z,https://twitter.com/search?q=lang%3Aen%20pokem...,that gon na yike cotton
4,Sat Jan 23 07:24:07 +0000 2021,929101255284285440,ItsAbouTimeJoey,Posted a Pokémon tier list. \n\nGonna lose som...,https://twitter.com/ItsAbouTimeJoey,Joey ⏱,https://twitter.com/ItsAbouTimeJoey/status/135...,2021-01-26T18:54:56.280Z,https://twitter.com/search?q=lang%3Aen%20pokem...,post pokémon tier list gon na lose follow tonight
...,...,...,...,...,...,...,...,...,...,...
576,Wed Oct 16 14:25:32 +0000 2019,719461958550822912,zaneflynt,imagine having real Pokémon opinions.. couldn'...,https://twitter.com/zaneflynt,hawke the nibblenib,https://twitter.com/zaneflynt/status/118447546...,2021-01-26T18:54:56.282Z,https://twitter.com/search?q=lang%3Aen%20pokem...,imagin real pokémon could
577,Fri Jul 14 16:46:29 +0000 2017,2267742470,Poijz,I liked a @YouTube video https://t.co/T31TIjz1...,https://twitter.com/Poijz,Dylan ポイ❄️,https://twitter.com/Poijz/status/8859033354753...,2021-01-26T18:54:56.282Z,https://twitter.com/search?q=lang%3Aen%20pokem...,I like youtub video http top 5 unpopular pokem...
578,Wed Oct 16 14:25:32 +0000 2019,719461958550822912,zaneflynt,imagine having real Pokémon opinions.. couldn'...,https://twitter.com/zaneflynt,hawke the nibblenib,https://twitter.com/zaneflynt/status/118447546...,2021-01-26T18:54:56.282Z,https://twitter.com/search?q=lang%3Aen%20pokem...,imagin real pokémon could
579,Sun Nov 03 02:45:47 +0000 2019,15121903,cheesegratersan,"me when it comes to ""Pokemon Opinions"": im a r...",https://twitter.com/cheesegratersan,hhhh,https://twitter.com/cheesegratersan/status/119...,2021-01-26T18:54:56.282Z,https://twitter.com/search?q=lang%3Aen%20pokem...,come pokemon opinion im cant get enough


Creación del bag of words y de la variable textsBoWOpinions.

In [84]:
bagOfWordsModelOpinions = TfidfVectorizer()
bagOfWordsModelOpinions.fit(preprocessedDataOpinions['processed_text'])
textsBoWOpinions= bagOfWordsModelOpinions.transform(preprocessedDataOpinions['processed_text'])
print("Finished")

Finished


In [85]:
textsBoWOpinions.shape

(581, 1885)

Predicción realizada:

In [86]:
X_testOpinions = textsBoWOpinions

predictions = svc.predict(X_test)

print(predictions)

[-1 -1 -1 -1 -1 -1 -1 -1 -1 -1  0 -1  0 -1 -1 -1 -1 -1 -1  0  1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1  0 -1  0 -1  0  1  0 -1  0 -1 -1 -1  1 -1 -1
 -1 -1 -1  0 -1 -1 -1 -1  0 -1 -1 -1  0 -1 -1 -1 -1 -1  0  0  0  0  0 -1
 -1  0 -1  0  0  0  0 -1 -1  0 -1  0  0 -1 -1 -1  0 -1  0 -1  0  0 -1 -1
 -1 -1 -1 -1 -1 -1  0  0 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1  0  0  0
 -1 -1 -1 -1 -1 -1  0  0  0 -1  0 -1 -1 -1 -1 -1 -1 -1 -1 -1  1 -1 -1 -1
  0 -1 -1 -1  0 -1 -1 -1 -1  0  0  1 -1  0 -1 -1 -1 -1  1 -1  0 -1 -1 -1
 -1  1 -1 -1  0 -1 -1  0  0  0 -1 -1  0  0 -1 -1 -1 -1 -1  1 -1  0  1 -1
 -1 -1 -1 -1  0  0 -1 -1 -1  0 -1 -1 -1 -1  0  0 -1 -1 -1 -1  0 -1 -1 -1
 -1 -1 -1 -1 -1  0 -1 -1  0  0 -1 -1  0  1  0  0  0 -1 -1 -1  0 -1  0  0
  0 -1 -1 -1 -1 -1 -1 -1 -1 -1  0 -1 -1 -1  0 -1 -1 -1 -1 -1 -1  0 -1 -1
 -1 -1 -1  0  0  0 -1 -1 -1  0 -1 -1  0 -1 -1 -1 -1 -1  1  0 -1  0 -1 -1
 -1 -1  0 -1 -1 -1 -1 -1 -1 -1  0 -1 -1  0 -1 -1 -1 -1  0  0 -1 -1 -1 -1
 -1  0  0  1 -1 -1 -1  0 -1 -1 -1 -1 -1 -1  0 -1 -1

**Valoración Manual**

Por ultimo explicaremos la simulación de nuestro sistema de valoración manual. Para ello hemos creado un formulario que permite introducir al usuario una puntuación y ademas un comentario sobre un Pokemon en concreto. Las secciones del formulario son:

- Pokemon: Pokemon a valorar.

- Puntuación: Valor seleccionable de 1 a 5 siendo 5 una valoración muy positiva y 1 una valoración muy negativa.

- Comentario: Comentario adicional sobre el Pokemon valorado.

In [87]:
Pokemon = 'Raichu' #@param {type:"string"}
Puntuacion= '5' #@param ["1", "2", "3","4","5"]
Comentario = 'Me parece un gran pokemon con una potencia increible' #@param {type:"string"}

print(Pokemon)
print(Puntuacion)
print(Comentario)

Raichu
5
Me parece un gran pokemon con una potencia increible


# BIBLIOGRAFÍA

[Uso de phantomBuster para la generación del csv sobre una busqueda de Twitter](https://phantombuster.com/)

[Información sobre sklearn.metrics](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html)

[Información sobre valoración automática en Python](https://www.arsys.es/blog/analisis-sentimientos-python-jupyter-notebooks/)

[GitHub con Notebook de Recomendación de peliculas y Notebook de Analisis de Sentimientos](https://github.com/adrseara/abp_notebooks)